# Bcrypt

## Inicializacion (Importacion de tablas)

In [ ]:
# Descarga las constantes de Blowfish (S-boxes y P-array) desde la web de Bruce Schneier
! curl -fLsS https://www.schneier.com/wp-content/uploads/2015/12/constants-2.txt -o constants.txt
! echo "Bajado constants.txt (tamaño): $(wc -c < constants.txt) bytes"


Bajado constants.txt (tamaño): 14215 bytes


In [ ]:
import re, textwrap, os, sys

# Lee el archivo descargado
with open("constants.txt", "r", encoding="utf-8", errors="ignore") as f:
    src = f.read()

# Extrae todos los literales hex (acepta 0x... o 0X..., con o sin sufijo L)
hex_tokens = re.findall(r'0[xX][0-9A-Fa-f]+L?', src)

if len(hex_tokens) < (256*4 + 18):
    raise RuntimeError(f"No se extrajeron suficientes constantes: {len(hex_tokens)}")

# Convierte a enteros (quitando 'L' si existe) y fuerza a 32 bits sin signo
vals = []
for h in hex_tokens:
    if h.endswith(('L','l')):
        h = h[:-1]
    vals.append(int(h, 16) & 0xFFFFFFFF)

# El archivo de Schneier ya viene "como cuatro s_boxes & un p_array" en ese orden.
# Tomamos los primeros 256 para S0, siguientes 256 S1, luego 256 S2, luego 256 S3, y por último 18 para P.
S0 = vals[0:256]
S1 = vals[256:512]
S2 = vals[512:768]
S3 = vals[768:1024]
P  = vals[1024:1024+18]

# Validaciones rápidas
assert len(S0) == len(S1) == len(S2) == len(S3) == 256, "S-box mal dimensionada"
assert len(P) == 18, "P-array mal dimensionado"
assert P[0] == 0x243F6A88, "Primer valor de P no coincide con el esperado (0x243F6A88)"

# Escribe el módulo Python listo para importar
with open("blowfish_tables.py", "w", encoding="utf-8") as out:
    def dump_list(name, arr):
        out.write(f"{name} = [\n")
        line = []
        for i, v in enumerate(arr):
            line.append(f"0x{v:08X}")
            if (i+1) % 8 == 0:
                out.write("  " + ", ".join(line) + ",\n")
                line = []
        if line:
            out.write("  " + ", ".join(line) + ",\n")
        out.write("]\n\n")

    out.write("# Auto-generado desde constants.txt de Bruce Schneier\n")
    out.write("# Contiene P (18) y S0..S3 (256 c/u) para Blowfish, en Python 3.\n\n")
    dump_list("P", P)
    dump_list("S0", S0)
    dump_list("S1", S1)
    dump_list("S2", S2)
    dump_list("S3", S3)

print("Archivo blowfish_tables.py generado OK.")


Archivo blowfish_tables.py generado OK.


In [ ]:
from blowfish_tables import P, S0, S1, S2, S3
print(len(P), P[0])
print(len(S0), len(S1), len(S2), len(S3))


18 608135816
256 256 256 256


## Implementacion Bcrypt

In [ ]:
import os

db = {} # Contraseñas hasheadas guardadas

def to_bytes(s: str):
    return s.encode('utf-8')

def to_str(b : bytes):
    return b.decode('utf-8')

def hex8(b : bytes):
    return b.hex()


In [ ]:
def F(x):
    # Dividir en 4 bytes
    a = (x >> 24) & 0xFF
    b = (x >> 16) & 0xFF
    c = (x >> 8) & 0xFF
    d = x & 0xFF
    t = (S0[a] + S1[b]) & 0xffffffff
    t = (t ^ S2[c]) & 0xffffffff
    t = (t + S3[d]) & 0xffffffff
    return t

In [ ]:
def split64_be(block8: bytes):
    # Divide 8 bytes big-endian en dos enteros de 32 bits (L, R)
    if len(block8) != 8:
        raise ValueError("El bloque no tiene exactamente 8 bytes.")
    L = int.from_bytes(block8[:4], byteorder='big')
    R = int.from_bytes(block8[4:], byteorder='big')
    return L, R

def join64_be(L, R):
    # Junta dos enteros de 32 bits (L, R) en 8 bytes big-endian
    return L.to_bytes(4, 'big') + R.to_bytes(4, 'big')

In [ ]:
m = b'\x01\x02\x03\x04\x05\x06\x07\x08'
L, R = split64_be(m)
print("L:", hex(L), "R:", hex(R))
print("Reconstruido:", join64_be(L, R).hex())


L: 0x1020304 R: 0x5060708
Reconstruido: 0102030405060708


In [ ]:
cnt_calls = 0

def encrypt_block(block8, P, S0, S1, S2, S3):
    global cnt_calls
    cnt_calls += 1
    L, R = split64_be(block8)

    for i in range(16):
        L = (L ^ P[i]) & 0xffffffff
        R = (R ^ F(L)) & 0xffffffff
        L, R = R, L
    L, R = R, L
    R = (R ^ P[16]) & 0xffffffff
    L = (L ^ P[17]) & 0xffffffff

    return join64_be(L, R)

In [ ]:
out1 = encrypt_block(b'\x00'*8, P, S0, S1, S2, S3)
out2 = encrypt_block(b'\x00\x00\x00\x00\x00\x00\x00\x01', P, S0, S1, S2, S3)
print(out1.hex())
print(out2.hex())


706d9fcc1792d23a
8bc745f9c197ac40


In [ ]:
def expand_key_keyonly(state, key_bytes: bytes):
    while key_bytes.__len__() % 4 != 0:
        key_bytes += b'\x00'
    key_words = []
    for i in range(0, key_bytes.__len__(), 4):
        word = key_bytes[i:i+4]
        key_words.append(word)
    for i in range(18):
        state["P"][i] = (state["P"][i] ^ int.from_bytes(key_words[i % key_words.__len__()], byteorder='big')) & 0xffffffff


    block = b'\x00' * 8
    for i in range(0, 18, 2):
        block = encrypt_block(block, state["P"], state["S0"], state["S1"], state["S2"], state["S3"])
        L, R = split64_be(block)
        state["P"][i] = L
        state["P"][i + 1] = R

    for i in range(0, 256, 2):
        block = encrypt_block(block, state["P"], state["S0"], state["S1"], state["S2"], state["S3"])
        L, R = split64_be(block)
        state["S0"][i] = L
        state["S0"][i + 1] = R

    for i in range(0, 256, 2):
        block = encrypt_block(block, state["P"], state["S0"], state["S1"], state["S2"], state["S3"])
        L, R = split64_be(block)
        state["S1"][i] = L
        state["S1"][i + 1] = R

    for i in range(0, 256, 2):
        block = encrypt_block(block, state["P"], state["S0"], state["S1"], state["S2"], state["S3"])
        L, R = split64_be(block)
        state["S2"][i] = L
        state["S2"][i + 1] = R

    for i in range(0, 256, 2):
        block = encrypt_block(block, state["P"], state["S0"], state["S1"], state["S2"], state["S3"])
        L, R = split64_be(block)
        state["S3"][i] = L
        state["S3"][i + 1] = R



In [ ]:
cursor = 0
def take8(data_bytes: bytes) -> bytes:
    global cursor
    if len(data_bytes) == 0:
        return b'\x00' * 8
    if cursor + 8 <= len(data_bytes):
        ret = data_bytes[cursor:cursor+8]
        cursor += 8
        if cursor == len(data_bytes):
            cursor = 0
        return ret
    else:
        first = data_bytes[cursor: ]
        needs = 8 - len(first)
        second = data_bytes[0:needs]
        cursor = needs
        return first + second

def xor8(a: bytes, b: bytes) -> bytes:
    return bytes(x ^ y for x, y in zip(a, b))

In [ ]:
def expand_key_data_key(state, data_bytes: bytes, key_bytes: bytes):
    # key_bytes seria la contrasena, data_bytes la salt
    global cursor
    cursor = 0

    while key_bytes.__len__() % 4 != 0:
        key_bytes += b'\x00'
    key_words = []
    for i in range(0, key_bytes.__len__(), 4):
        word = key_bytes[i:i+4]
        key_words.append(word)

    for i in range(18):
        state["P"][i] = (state["P"][i] ^ int.from_bytes(key_words[i % key_words.__len__()], byteorder='big')) & 0xffffffff


    block = b'\x00' * 8
    for i in range(0, 18, 2):
        block_in = xor8(block, take8(data_bytes))
        block_out = encrypt_block(block_in, state["P"], state["S0"], state["S1"], state["S2"], state["S3"])
        L, R = split64_be(block_out)
        state["P"][i] = L
        state["P"][i + 1] = R
        block = block_out

    for name in ("S0", "S1", "S2", "S3"):
        for i in range(0, 256, 2):
            block_in = xor8(block, take8(data_bytes))
            block_out = encrypt_block(block_in, state["P"], state["S0"], state["S1"], state["S2"], state["S3"])
            L, R = split64_be(block_out)
            state[name][i] = L
            state[name][i + 1] = R
            block = block_out

In [ ]:
state = {}

def restart_state():
    global state
    state["P"] = P.copy()
    state["S0"] = S0.copy()
    state["S1"] = S1.copy()
    state["S2"] = S2.copy()
    state["S3"] = S3.copy()

restart_state()
print(state["P"][:6])
cnt_calls = 0

expand_key_keyonly(state, b'\x00'*32)

print(state["P"][:6])

print(f'Calls to encrypt_block: {cnt_calls}')





[608135816, 2242054355, 320440878, 57701188, 2752067618, 698298832]
Number of words 8
[1886232524, 395498042, 767153940, 2523796537, 2887886701, 2200234376]
Calls to encrypt_block: 521


In [ ]:
restart_state()
print(state["P"][:6])
expand_key_data_key(state, b'\x00'*16, b'\x00'*8)
print(state["P"][:6])
print(f'Calls to encrypt_block: {cnt_calls}')

[608135816, 2242054355, 320440878, 57701188, 2752067618, 698298832]
[1886232524, 395498042, 767153940, 2523796537, 2887886701, 2200234376]
Calls to encrypt_block: 1042


In [ ]:
def eks_setup(password_bytes, salt_bytes, cost):
    # Estado inicial
    state = {}
    state["P"] = P.copy()
    state["S0"] = S0.copy()
    state["S1"] = S1.copy()
    state["S2"] = S2.copy()
    state["S3"] = S3.copy()

    expand_key_data_key(state, salt_bytes, password_bytes)

    for _ in range(1 << cost):
        expand_key_keyonly(state, password_bytes)

    return state

In [ ]:
# Pruebas de eks_setup

import time
password = b"abc"
salt = b"\x00"*16

time_start = time.time()
state = eks_setup(password, salt, 4)
time_end = time.time()
print(f'Cost: {4}, Time: {time_end - time_start}')

time_start = time.time()
state = eks_setup(password, salt, 5)
time_end = time.time()
print(f'Cost: {5}, Time: {time_end - time_start}')





Cost: 4, Time: 0.14083504676818848
Cost: 5, Time: 0.26803159713745117


In [ ]:

def magic64(state):
    magic = b"OrpheanBeholderScryDoubt"
    B = [magic[0:8], magic[8:16], magic[16:24]]
    for i in range(3):
        for _ in range(64):
            B[i] = encrypt_block(B[i], state["P"], state["S0"], state["S1"], state["S2"], state["S3"])

    hash24 = B[0] + B[1] + B[2]
    return hash24

def format_bcrypt_string(cost, salt, hash24):
    return f"$2b${cost}${hex8(salt)}{hex8(hash24)}"

def apply_bcrypt(password, salt, cost = 10):
    state = eks_setup(password.encode('utf-8'), salt, cost)
    hash24 = magic64(state)
    stored = format_bcrypt_string(cost, salt, hash24)
    return stored



def store_password(password, user):
    salt = os.urandom(16)
    hashed_password = apply_bcrypt(password, salt)
    db[user] = hashed_password

In [ ]:
# Chat GPT test

# ===========================
# TEST HARNESS (con tu formato HEX)
# ===========================
import os, time, hmac, re
from binascii import unhexlify, hexlify

# --- Utilidades pequeñas ---

def parse_stored_hex(stored: str):
    """
    Tu formato actual: "$2b$<cost>$<salt_hex><hash_hex>"
    salt: 16 bytes -> 32 hex
    hash: 24 bytes -> 48 hex
    """
    m = re.match(r'^\$2b\$(\d+)\$([0-9a-fA-F]+)$', stored)
    if not m:
        raise ValueError(f"Formato inesperado: {stored}")
    cost = int(m.group(1))
    tail = m.group(2)
    if len(tail) != (32 + 48):
        raise ValueError(f"Largo inesperado de hex total (salt+hash): {len(tail)} (esperado 80)")
    salt_hex = tail[:32]
    hash_hex = tail[32:]
    salt = unhexlify(salt_hex)
    hash24 = unhexlify(hash_hex)
    return cost, salt, hash24

def verify_password_hex(db, user: str, password: str) -> bool:
    """
    Recalcula con el mismo salt y cost, compara el string completo ($2b$...$<salt_hex><hash_hex>)
    """
    if user not in db:
        return False
    stored = db[user]
    cost, salt, _hash24 = parse_stored_hex(stored)
    recomputed = apply_bcrypt(password, salt, cost)
    # comparación en tiempo constante
    return hmac.compare_digest(stored.encode(), recomputed.encode())

def bench_apply(password: str, salt: bytes, cost: int = 10, repeat: int = 1):
    """
    Mide el tiempo de apply_bcrypt con parámetros dados.
    """
    starts = time.time()
    out = None
    for _ in range(repeat):
        out = apply_bcrypt(password, salt, cost)
    elapsed = time.time() - starts
    print(f"[bench] cost={cost}, repeat={repeat}, t={elapsed:.4f}s  → {out[:40]}...")
    return elapsed, out

# --- Tests básicos de cifrado de bloque (sanity) ---

def test_encrypt_block_sanity():
    print("== Test encrypt_block sanity ==")
    m1 = b'\x00'*8
    m2 = b'\x00\x00\x00\x00\x00\x00\x00\x01'
    c1 = encrypt_block(m1, P.copy(), S0.copy(), S1.copy(), S2.copy(), S3.copy())
    c2 = encrypt_block(m2, P.copy(), S0.copy(), S1.copy(), S2.copy(), S3.copy())
    print(" c1:", c1.hex())
    print(" c2:", c2.hex())
    assert c1 != c2, "Cambiar 1 bit del bloque debería cambiar fuertemente la salida"
    print(" OK\n")

# --- Tests de EKS y magic64 ---

def test_eks_and_magic():
    print("== Test Eks + magic64 ==")
    pw = "abc"
    salt = b"\x00"*16
    cost = 4

    # arma estado y hash24
    st = eks_setup(pw.encode(), salt, cost)
    h24 = magic64(st)
    print(" hash24(len=24):", h24.hex(), len(h24))
    assert len(h24) == 24, "hash24 debe tener 24 bytes"
    # sensibilidad a salt
    st2 = eks_setup(pw.encode(), salt[:-1]+b"\x01", cost)
    h24b = magic64(st2)
    assert h24 != h24b, "Cambiar el salt debe cambiar el hash24"
    print(" OK\n")

# --- Tests de store/verify con tu formato actual ---

def test_store_and_verify():
    print("== Test store_password / verify_password_hex (formato HEX actual) ==")
    # limpia DB
    db.clear()

    # usuario 1
    user1 = "ana"
    pw1 = "hola123"
    store_password(pw1, user1)
    print(" stored[ana]:", db[user1])
    assert verify_password_hex(db, user1, pw1), "Debería verificar TRUE con la contraseña correcta"
    assert not verify_password_hex(db, user1, "otra"), "Debería verificar FALSE con contraseña incorrecta"

    # usuario 2 (misma contraseña → debe quedar diferente por salt distinto)
    user2 = "luis"
    pw2 = "hola123"
    store_password(pw2, user2)
    print(" stored[luis]:", db[user2])
    assert db[user1] != db[user2], "Misma contraseña con salt distinto → cadenas almacenadas distintas"

    print(" OK\n")

# --- Test de escalado con cost ---

def test_cost_scaling():
    print("== Test escalado con cost ==")
    pw = "password!"
    salt = os.urandom(16)
    t4, _ = bench_apply(pw, salt, cost=4, repeat=1)
    t5, _ = bench_apply(pw, salt, cost=5, repeat=1)
    # debería ser aproximadamente 2x
    ratio = (t5 / t4) if t4 > 0 else 0
    print(f" ratio t5/t4 ≈ {ratio:.2f}")
    assert ratio > 1.5, "Esperaba que cost=5 sea ~2x cost=4 (±)"
    print(" OK\n")

# ===========================
# Ejecuta los tests
# ===========================
try:
    test_encrypt_block_sanity()
    test_eks_and_magic()
    test_store_and_verify()
    test_cost_scaling()
    print("✅ TODOS LOS TESTS PASARON con tu formato HEX actual.")
except AssertionError as e:
    print("❌ ASSERTION:", e)
except Exception as ex:
    print("❌ ERROR:", type(ex).__name__, ex)


== Test encrypt_block sanity ==
 c1: 706d9fcc1792d23a
 c2: 8bc745f9c197ac40
 OK

== Test Eks + magic64 ==
 hash24(len=24): 458d973febff8e2f3e3683ee42daa828ecbeb06c6ff5100d 24
 OK

== Test store_password / verify_password_hex (formato HEX actual) ==
 stored[ana]: $2b$10$b31b6cb9a69b302242a4224c981d94ad894532842ecf993e10c44996daa60cd1a548c571a86c0a20
 stored[luis]: $2b$10$1c23c01fc096a274ea5ce258075bbfa22aef163864d1b9de232a359deafe1e1a31c10160d11ab13b
 OK

== Test escalado con cost ==
[bench] cost=4, repeat=1, t=0.1417s  → $2b$4$67d01b4cbeb101d4af2234b5bff220bf3d...
[bench] cost=5, repeat=1, t=0.3912s  → $2b$5$67d01b4cbeb101d4af2234b5bff220bf7e...
 ratio t5/t4 ≈ 2.76
 OK

✅ TODOS LOS TESTS PASARON con tu formato HEX actual.


In [ ]:
store_password("constrasena", 'luis')

# Scrypt